<img src="http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png" align=left>
<img src="http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png" align=left>

In [1]:
# Run this cell to setup data path
import os

datapath = os.getcwd()
if datapath.find('databricks') != -1:
    ACCESS_KEY = "AKQQ"
    SECRET_KEY = "YJbbm"
    AWS_BUCKET_NAME = "nyc"
    datapath = "s3a://%s:%s@%s/" %(ACCESS_KEY, SECRET_KEY, AWS_BUCKET_NAME)

# Spark Lab: Word Count

This lab will build on the techniques covered in the Spark intro to develop a simple word count application. In this lab, we will write code that calculates the most common words in a text.
 
the goal is to learn the distribution of letters in the most popular words in a corpus
This could also be scaled to find the most common words in Wikipedia.

** Here're the suggested steps: **

* *Step 1:* Creating a base RDD from a python list
* *Step 2:* Text cleaning
* *Step 3:* Counting unique words
* *Step 4:* Putting them together
* *Step 5:* Writing a self-contained applications

## Step 1: Creating a pair RDD

**1.1** We'll start by generating a base RDD from the file `pyspark_1/README.md` by using the `sc.textFile()` method. Then we'll print out the type of the base RDD.

In [2]:
# TODO: create a spark RDD `lineRDD` with lines.

filePath = os.path.join(datapath, "pyspark_1/README.md")

lineRDD = sc.textFile(filePath)
print type(lineRDD)

<class 'pyspark.rdd.RDD'>


**1.2** Remember all transformations in Spark are lazy, in that they do not compute their results right away until an action requires a result to be returned to the driver program. To make errors easier to localize, we can apply an action to `lineRDD` to see if it works properly.

Usually, `collect()` method is **NOT** recommended as it will collect all the data to the driver node. Now apply `take(5)` to the `lineRDD` you just created and see if the file has been load successfully.

In [3]:
# TODO: test lineRDD by applying an action `take(5)` to it.

# Your code goes here
lineRDD.take(5)

[u'# Apache Spark',
 u'',
 u'Spark is a fast and general cluster computing system for Big Data. It provides',
 u'high-level APIs in Scala, Java, Python, and R, and an optimized engine that',
 u'supports general computation graphs for data analysis. It also supports a']

## Step 2: Text cleaning

**2.1** Before moving to further analysis, we want to clean the text a little bit. Here're some issues that you may want to address:

* All punctuation should be removed.
* All words should be in lower case so the same words of different cases are not considered to be different.
* All leading or trailing spaces should be removed.

Define the function cleanText to do the work. You may want to use the Python `re` and/or `string` module to remove punctuation. Then test `cleanTest()` with `linesRDD`.

In [4]:
import re, string

def cleanText(text):
    """
    Removes punctuation, change to lower case, and strips white spaces
    
    :param text(str): A string.
    :return: the cleaned string
    """
    
    # Your code goes here
    cleaned_text = re.sub("\W", " ", text).lower().strip()
    return cleaned_text

if __name__ == "__main__":
    # Test cleanText() with texts in lines
    lines = lineRDD.takeSample(False, 3)
    print cleanText(lines[0])
    print cleanText(lines[1])
    print cleanText(lines[2])


dev run tests



**2.2** Now create another RDD with the name `lineCleaned` by applying the `cleanText()` to `lineRDD`. To test the result, use `take(5)` to print the first 5 lines.

In [5]:
# TODO: create lineCleaned and then test it with take(5)

lineCleaned = lineRDD.map(cleanText) #<fill in>
print lineCleaned.take(5)

[u'apache spark', u'', u'spark is a fast and general cluster computing system for big data  it provides', u'high level apis in scala  java  python  and r  and an optimized engine that', u'supports general computation graphs for data analysis  it also supports a']


## Step 3: Counting unique words

**3.1** To count the number of occurence of each word, we can:

* Create an RDD `wordRDD` by splitting the lines to words using `flatMap()`.
* Create a pair RDD `wordPair` with value of each word equals 1.
* Create another pair RDD `wordCount` by aggregrating the values of the same word in `wordPair`.

In [6]:
# TODO: first create `wordRDD` and then `wordPair`.

wordRDD = lineCleaned.flatMap(lambda line: line.split()) #<fill in>
wordPair = wordRDD.map(lambda word: (word, 1)) #<fill in>

print wordPair.take(5)

[(u'apache', 1), (u'spark', 1), (u'spark', 1), (u'is', 1), (u'a', 1)]


In [7]:
# TODO: create `wordCount` by adding values of the same words in `wordPair`.

# Hint: you can use either `groupByKey()` followed by `mapValues()` or `reduceByKey()`

from operator import add
wordCount = wordPair.reduceByKey(add) #<fill in>

print wordCount.take(5)

[(u'please', 3), (u'through', 1), (u'computation', 1), (u'1', 1), (u'mesos', 1)]


**3.2** Usually the frequency reflects how important a word is to a document. We can view the most frequent words by using the [`takeOrdered()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.takeOrdered) action; however, since the elements of the RDD are pairs, we need a custom sort function that sorts using the value part of the pair.

In [8]:
# TODO: use `takeOrdered()` to get the top 10 most frequent word and their counts

frequentWord = wordCount.takeOrdered(10, key=lambda kv: -kv[1]) #<fill in>
print frequentWord

[(u'spark', 33), (u'the', 24), (u'to', 16), (u'for', 14), (u'run', 13), (u'apache', 13), (u'and', 12), (u'org', 11), (u'you', 9), (u'a', 9)]


**3.3** (Optional) You may notice that some of the most frequent words are stopwords - words which do not contain important significance. Sometimes you may want to remove the stopwords since they usually do not carry much useful information. 

If you decide to do so, use the `stopwords.txt` file in the directory `pyspark_1`. Then remove the stopwords by applying a filter to the RDD you just created.

In [9]:
# TODO: recreate `wordCount` by removing the stopwords. The filter can be applied to different places.

STOPWORDS_PATH = os.path.join(datapath, "pyspark_1/stopwords.txt")

cachedStopWords = sc.broadcast(sc.textFile(STOPWORDS_PATH).collect())

wordCount_removed = wordCount.filter(lambda kv: kv[0] not in cachedStopWords.value) #<fill in>
print wordCount_removed.takeOrdered(10, key=lambda kv: -kv[1]) #<fill in>

[(u'spark', 33), (u'run', 13), (u'apache', 13), (u'org', 11), (u'building', 8), (u'example', 8), (u'hadoop', 7), (u'maven', 6), (u'http', 6), (u'using', 6)]


## Step 4: Putting them together

Now we can define a function `wordCount()` to count the frequencies of unique words in an RDD of text by putting all the transformations from step 1 throuth step 3 together.

In [10]:
# TODO: complete the wordCount() function

def wordCount(lineRDD):
    """
    Creates a pair RDD with word counts from an RDD of text.
    :param wordListRDD(pyspark.rdd.RDD): An RDD of text.
    :return: Return the count of each unique word in this RDD as a pair RDD
    """
    wordCounts = (lineRDD
                  .map(cleanText)
                  .flatMap(lambda line: line.split())
                  .filter(lambda word: word not in cachedStopWords.value)
                  .map(lambda word: (word, 1))
                  .reduceByKey(add))
    return wordCounts


In [11]:
# TODO: test the function `wordCount()` with `takeOrdered()`

# Your code goes here
wordCount(lineRDD).takeOrdered(10, key=lambda kv: -kv[1])

[(u'spark', 33),
 (u'run', 13),
 (u'apache', 13),
 (u'org', 11),
 (u'building', 8),
 (u'example', 8),
 (u'hadoop', 7),
 (u'maven', 6),
 (u'http', 6),
 (u'using', 6)]

## Step 5: Writing a self-contained applications

Lets write a self-contained application using the Spark API. The app should read a text file and return the word counts.

Note that we need to create a `SparkContext` object, which tells Spark how to access a cluster.

In [ ]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("pysparkWordCount")
sc = SparkContext(conf=conf)

Write your `wordcount.py` file from scratch. When you finish, use `spark-submit wordcount.py <text file>` to run it from command line.